# Integrate a regression tree model to classify a person's position based on landmark distances
- Collect Training Data: Capture images with known position labels (e.g., "Standing", "Push-up Up", "Push-up Down", "Lying Down").
- Extract Features: Compute landmark distances (e.g., shoulder-to-hip, hip-to-knee, elbow angle, etc.).
- Train a Decision Tree: Use scikit-learn's DecisionTreeClassifier or DecisionTreeRegressor to classify or predict positions.
- Integrate into Real-Time Detection: Use the trained model to classify the current pose.

---
# Embeddings in a CSV file

---

In [21]:
import cv2
import mediapipe as mp
import csv
import numpy as np
import os
import math

In [22]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Directory containing labeled folders with images
base_folder = "./training_datasets/push-ups_images"  # Change to your folder path

# Create CSV file and write headers
output_folder ="./models"
csv_filename = "push-up_landmark_positions.csv"
headers = ["shoulder_hip_dist", "hip_knee_dist", "elbow_angle", "label"]

with open(os.path.join(output_folder,csv_filename), "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    
    for label in os.listdir(base_folder):
        label_folder = os.path.join(base_folder, label)
        if not os.path.isdir(label_folder):
            continue
        
        for image_name in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_name)
            
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get key landmarks
                shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
                hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
                knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
                elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
                wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
                
                # Compute distances
                shoulder_hip_dist = np.linalg.norm([shoulder.x - hip.x, shoulder.y - hip.y])
                hip_knee_dist = np.linalg.norm([hip.x - knee.x, hip.y - knee.y])
                
                # Compute elbow angle
                def calculate_angle(a, b, c):
                    """Calculate the angle between three points"""
                    ba = np.array([a.x - b.x, a.y - b.y])
                    bc = np.array([c.x - b.x, c.y - b.y])
                    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
                
                elbow_angle = calculate_angle(shoulder, elbow, wrist)
                
                # Save to CSV
                print( [shoulder_hip_dist, hip_knee_dist, elbow_angle, label] )
                writer.writerow([shoulder_hip_dist, hip_knee_dist, elbow_angle, label])
                print(f"Data saved: {shoulder_hip_dist}, {hip_knee_dist}, {elbow_angle}, {label}")
            
            cv2.imshow("Pose Capture", image)
            if cv2.waitKey(500) & 0xFF == ord('q'):
                break
    
cv2.destroyAllWindows()

W0000 00:00:1743587682.040880   10010 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743587682.324913   10010 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


[0.23901070428335272, 0.17891223845862922, 176.71749311356987, 'position_up']
Data saved: 0.23901070428335272, 0.17891223845862922, 176.71749311356987, position_up
[0.1891974593360051, 0.16304250425324465, 169.88347601892454, 'position_up']
Data saved: 0.1891974593360051, 0.16304250425324465, 169.88347601892454, position_up
[0.14490319999544696, 0.12779556589018212, 179.17744486051618, 'position_up']
Data saved: 0.14490319999544696, 0.12779556589018212, 179.17744486051618, position_up
[0.2107844386465685, 0.20438144971703723, 174.2199571839313, 'position_up']
Data saved: 0.2107844386465685, 0.20438144971703723, 174.2199571839313, position_up
[0.1714154898415672, 0.17607206829545774, 148.7106169707099, 'position_up']
Data saved: 0.1714154898415672, 0.17607206829545774, 148.7106169707099, position_up
[0.07825545438543634, 0.11865901981625623, 92.70856677458079, 'position_up']
Data saved: 0.07825545438543634, 0.11865901981625623, 92.70856677458079, position_up
[0.12051826464895456, 0.0605

In [ ]:
from ACV_Classifier_Fonctions import MediaPipe
import os
import cv2
import csv

mp=MediaPipe()

# Directory containing labeled folders with images
base_folder = "./training_datasets/push-ups_images"  # Change to your folder path

# Create CSV file and write headers
output_folder ="./models"
csv_filename = "push-up_landmark_positions_test.csv"
headers=[
    "left_shoulder_hip_dist", "left_hip_knee_dist", "left_elbow_angle",
    "right_shoulder_hip_dist", "right_hip_knee_dist", "right_elbow_angle",
    "label"
    ]


with open(os.path.join(output_folder,csv_filename), "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    
    for label in os.listdir(base_folder):
        label_folder = os.path.join(base_folder, label)
        if not os.path.isdir(label_folder):
            continue
        
        for image_name in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_name)
            
            image = cv2.imread(image_path)
            if image is None:
                continue

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            image_data=mp.get_distances_and_angles(image_rgb)
            csv_data=[]
            for header in headers:
                try:
                    csv_data.append(image_data[header])
                except:
                    None
            csv_data.append(label)
            
            #Save to CSV
            print(csv_data)
            writer.writerow(csv_data)
            print(f"Data saved:",csv_data)
            
            cv2.imshow("Pose Capture", image)
            if cv2.waitKey(500) & 0xFF == ord('q'):
                break
    
cv2.destroyAllWindows()

W0000 00:00:1743589403.155857   11255 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743589403.207943   11255 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/romain/anaconda3/envs/ACV/lib/python3.12/site-packages/cv2/qt/plugins"


[0.23901070428335272, 0.17891223845862922, 176.71749311356987, 0.21078884206869725, 0.18491114875379538, 162.92472455945023, 'position_up']
Data saved: [0.23901070428335272, 0.17891223845862922, 176.71749311356987, 0.21078884206869725, 0.18491114875379538, 162.92472455945023, 'position_up']
[0.1891974593360051, 0.16304250425324465, 169.88347601892454, 0.2320347521938916, 0.15395177258282622, 171.25025903735286, 'position_up']
Data saved: [0.1891974593360051, 0.16304250425324465, 169.88347601892454, 0.2320347521938916, 0.15395177258282622, 171.25025903735286, 'position_up']
[0.1891974593360051, 0.16304250425324465, 169.88347601892454, 0.2320347521938916, 0.15395177258282622, 171.25025903735286, 'position_up']
Data saved: [0.1891974593360051, 0.16304250425324465, 169.88347601892454, 0.2320347521938916, 0.15395177258282622, 171.25025903735286, 'position_up']
[0.14490319999544696, 0.12779556589018212, 179.17744486051618, 0.23194923142674487, 0.08991707237619731, 178.36446068984057, 'positi

---
# Classification

---

In [6]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import joblib
import os

In [7]:
# Load collected data
data = pd.read_csv(os.path.join(output_folder,csv_filename))  # Columns: distances + labels
# data = data.rename(columns={'pushups_up':'label', 'image_001.jpg':'name'})
# data = data.drop(columns=['name'])
data

,shoulder_hip_dist,hip_knee_dist,elbow_angle,label
0,0.239011,0.178912,176.717493,position_up
1,0.189197,0.163043,169.883476,position_up
2,0.144903,0.127796,179.177445,position_up
3,0.210784,0.204381,174.219957,position_up
4,0.171415,0.176072,148.710617,position_up
5,0.078255,0.118659,92.708567,position_up
6,0.120518,0.060559,105.828327,position_up
7,0.111322,0.106019,165.656647,position_up
8,0.155676,0.196791,170.171038,position_up
9,0.164932,0.187251,29.809285,position_up


In [11]:
X = data.drop(columns=['label'])  # Features: distances
y = data['label']  # Target: position labels

# Train classifier
clf = DecisionTreeClassifier()
clf.fit(X, y)

# Save model
joblib.dump(clf,os.path.join(output_folder,"push-up_classifier.pkl"))

['./models/push-up_classifier.pkl']